# Model Mis-specification and Inverse Reinforcement Learning

Model Mis-specification  
비선형의 데이터를 liner model을 가지고 추론하는 경우.. 즉 데이터에 적합하지 않은 모델을 지정했다.

이 포스트에서는 inverse reinforcemnet learining 에 대해서 다룰 것이다. 이는 agent의 action을 관찰하여 그런 action을 이끌어내는 어떤 preference or beliefs를 추론하는 것이다. 이 포스트를 위해서 Owain Evans - modeling agents with probabilistic programs 책을 쓴 저자 중 하나이고 이 연구분야의 활동적인 연구자중 하나 - 와 함께 했다.

Owain 과 나는 AI가 인간의 가치를 배우는 방법으로 제안되었기 때문에 IRL에 특히 관심이 많습니다. 이는 결국에는 learining을 효율적으로 하고 인간의 value를 구현함으로써 agent를 파워풀 하게 해줄 것이고 더 넓은 scope에서 행동할 수 있게 할 것입니다.우리는 IRL 이 이를 고려하기에 적합하다고 생각하지만 잘못 설계한 모델은 IRL의 성능과 밀접한 관계가 있다고 생각합니다. IRL의 역할은 인간의 value를 추론하는 것이고 이는 reward function ro utility function으로 표현되어 집니다. 하지만 문제는 human value는 직접적으로 관찰되어지지 않습니다.


아래에서는 이러한 문제에 대해 자세히 설명합니다. 우리는 이러한 문제를 더 잘 이해함으로써 역 보강 학습에 종사하는 연구원들이 결과로 발생하는 잘못을 예측하고 해결할 수 있기를 바랍니다. 또한 특정 구체적 상황에서  model mis-specification으로 인한 문제를 고려하면 mis-specification에 대한 이전 게시물에서 지적한 일반적인 문제를 더 잘 해명 할 수 있다고 생각합니다.

# Specific Pitfalls for Inverse Reinforcement Learning

 “Latent Variables and Model Mis-specification”, 에서 Jacob 은 model mis-specification - true model 에는 적용되지 않지만 우리가 고려하고 있는것"에 대해 언급했습니다. 우리는 독자들에게 그 게시물을 먼저 읽도록 권합니다.하지만 우리는 아래를 독립적으로 읽을 수있게하려고 노력했습니다.
 
역 강화 학습과 관련하여 Model Mis-specification으로 인해 발생할 수있는 몇 가지 특정 문제를 볼 수 있습니다. 예를 들어 다음은 agent 에 대해 오해 할 수있는 사항이며 agent 의 가치에 대한 잘못된 추론을 유발할 수 있습니다.

- the actions of the agent :
    - agent가 특정 행동을 취할 수 있다고 생각하지만 실제로는 행동을 취하지 않는다고 판단되면 해당 값에 대해 이상한 추측을 할 수 있습니다 (예 : 행동을 취하지 않는 것에 높은 value 를 가지는 것). 또한, 우리의 데이터가 인간 행동의 비디오 인 경우, 우리는 프레임으로부터 행동을 인식하는 추가 추론 문제가 있습니다.

- The information available to the agent. 
    - agent가 우리가 생각하는 것보다 많은 정보에 액세스 할 수 있다면, (reward function의 관점에서) 우리에게 비이성적으로 보이는 계획은 agent 에게는 최적 일 수 있습니다. 다른 방향에서, agent가 우리가 생각하는 것보다 적은 정보를 가지고 있다면, 정보 부족으로 인해 A를 얻지 못했지만 A를 가치가 없다고 잘못 판단 할 수 있습니다.

- The long-term plans of the agent.
    - agent는 장기 목표를 달성하는 데 유용한 많은 action을 취할 수 있지만, 반드시 우리가 agent를 시간범위 이상으로 관찰할 필요는 없습니다. (즉 100 이라는 기간을 관찰해서 200 까지 추론하는 것이지 200을 관찰해서 200을 추론하는 것이 아니다) 따라서 정확한 value를 추론하는 것은 그러한 장기 목표를 추론 할 수 있도록 모델링 해야 합니다. 또한, long time horizons은 모델을 더 취약하게 만들 수있어 Model Mis-specification를악화시킵니다.
    
다른 오류도 있을 수 있습니다. 일반적인 지적은 Mis-specification Model 이 주어지면 agent의 value를 잘못 추정하는 것은 매우 자주 일어나는 일이라는 것입니다. optimization 을 함으로써 Mis-specification Model을 현재의 sample에 최대로 맞추려고 하는 것 뿐입니다.

이 글의 나머지 부분에서는 위의 각 측면 을 순서대로 다루며 — actions, information, and plans — , 대리인의 측면에 대한 Model Mis-specification가 왜곡 된 신념을 유발할 수있는 지에 대한 양적 모델과 질적 논증을 모두 제공합니다 및 행동. 먼저 역 보강 학습의 정의를 간략히 검토하고 관련 표기법을 소개합니다.

# Inverse Reinforcement Learning: Definition and Notations

역 강화 학습에서는 주어진 환경에서 행동을 취하는 에이전트를 모델링하려고합니다. 그러므로 우리는 <b>state space</b> S(agent와 environment가 가질 수 있는 state의 집합), <b>action space</b> A(agent가 취할 수 있는 action의 집합), 현재 state에서 다음 state로 갈 확률을 제공 하는 <b>transition function</b>  $T (s '\mid s, a)$ 를 가정합니다. 예를 들어, 자동차를 제어하는 AI 학습의 경우, state space은 자동차의 가능한 위치 및 방향이며, action space은 AI가 자동차에 보낼 수있는 제어 신호 세트 일 것이며, transition function은 자동차의 동력학 모델이되어야합니다. 앞의 세개의 집합인 튜플 (S, A, T) 는 MDP\R,라고 불리며 이는 보상 기능이없는 마르코프 결정 프로세스입니다. ( MDP\R 알려진 horizon 또는 discount factor $\gamma$을 갖지만 단순화를 위해이를 남겨 둘 것입니다.)

<img src="./image01.png">

그림 1 : IRL과 RL의 관계를 보여주는 다이어그램 (제공 : Pieter Abbeel의 IRL 슬라이드 )

IRL의 문제는 optimal policy  $ \pi^* : S -> A \ $이 주어졌을때 reward function R 을 추론하는 것입니다.  우리는 $ \pi ^ *$  을 (s, a) 로 부터 학습하고 $ \pi ^ * $ 로 부터 대응되는 action을 학습합니다. 일반적으로 이런 (s,a) 는 trajectory(single episode에서 나온 agent의 action, state)부터 뽑아 낼 수 있습니다.

$ (s_0, a_0), (s_1, a_1), \ldots, (s_n, a_n) $ 


자동차 예에서 이것은 원하는 운전 행동 (운전대가 조향 휠, 브레이크 등에 신호로 기록되는 곳)을 시연하는 노련한 운전자가 취한 행동과 일치합니다.  


MDP \R 과 어떤 trajectory 가 주어졌을때 IRL의 목표는 reward function 을 추론하는 것입니다. 베이지안 프레임 워크에서 어떤 R의 Specify 한 사전분포를 지정하면 밑처럼 나타낼 수 있습니다.


$P(R \mid s_{0:n},a_{0:n}) \propto P( s_{0:n},a_{0:n} \mid R) P(R) = P(R) \cdot \prod_{i=0}^n P( a_i \mid s_i, R)$


The likelihood $P(a_i \mid s_i, R)$ is just $\pi_R(s)[a_i]$, where $\pi_R$ is the optimal policy under the reward function R.

reward가 주어졌을때 optimal policy 를 계산하는 것은 non-trivial 합니다. 단순한 경우를 제외하고는 일반적으로 강화 학습을 사용하여 policy를 근사합니다 (그림 1 참조). policy는 일반적으로 잡음이있는 것으로 가정합니다 (예 : deterministic 한것 대신 softmax 사용). 사전분포를 지정하는 것의 어려움 때문에, optimal polciy, intergrating over reward function을 하는 것에 일반적으로 베이지안 방법을 사용합니다.

# Recognizing Human Actions in Data

IRL은 쉬운 데이터 가용성 때문에 인간 가치를 학습하는 유망한 접근법입니다. 감독 학습 (supervised learning)의 경우, 인간은 task 에 특화된 많은 레이블 된 sample을 생성해야합니다. 반면에, IRL 은 Unsupervised/ semi supervsied 접근입니다.

페이스 북의 사용자 행동 로그는 수많은 데이터를 제공한다.를 YouTube 비디오, 역사 책 및 문학은 실제 시나리오와 상상 된 시나리오 모두에서 인간 행동에 관한 데이터 모음입니다. 그러나 인간의 선호도에 유익한 많은 기존 데이터가 있지만 IRL에 데이터의 활용은 현재의 기술로는 어렵고 복잡한 작업이 될 것입니다.

Inferring Reward Functions from Video Frames

위에서 언급했듯이, IRL의 적용은 일반적으로 인간의 Policy $\pi^*$ 로 부터 관측된 sample 로 부터 reward function R을 추론합니다. environment는 $MDP\backslash R = (S,A,T)$ 일반적인 MDP 에서 R이 존재 하지 않는 형태이며 observation 은  인간의 policy로 부터관측된 $(s,a) \sim pi^*$ 입니다. 이 문제는 2가지를 가정하는데 (a) the environment’s dynamics T are given as part of the IRL problem, and (b) the observations are structured as “state-action” pairs.  

- (a) : environment's dynamics T (트랜지션 메트릭스)가 주어집니다. ( 자율주행의 경우.. 어떤 액션을 취하면 어디로 갈지는 명확합니다)
- (b) : observations 은 (s,a) 로 구성되어집니다.

데이터가 사람의 전문가가 차를 주차 할 때 나온 것이라면 이러한 가정은 합리적입니다. 운전자의 상태와 행동을 기록 할 수 있고 자동차 시뮬레이터를 사용할 수 있습니다. YouTube 동영상 또는 history books(페이스북 같은것인듯)의 데이터의 경우 가정이 실패합니다. The data is a sequence of partial observations : transition function T를 알 수 없으며 state와 action 을 분리 할 수 없습니다. 

(예를 들어보자면... face book 에서 좋아요를 누른다고 했을때 다음에 볼 게시물은 명확하지 않다. 즉 (a) 가정 불만족, youtube video 문제같은 경우는 state: video1,2,3,4 라고 하면 action: video1,2,3,4 이다 즉 state와 action을 분리 할 수 가 없다. (b) 가정을 불만족 ) 

결론은 text or video 로 부터 인간의 actions을 추론 하는 것은 정말 힘든 문제입니다.
 

<img src="./image00.png">

movie still :이 상황에서 수행중인 작업은 무엇입니까? 

구체적인 예로, 데이터가 비행기를 조종하는 두 명의 공동 조종사의 비디오라고 가정합니다. 연속적인 프레임은 각 시간 단계에서 state 에 대한 제한된 정보만을 제공합니다. 그래서 복잡한 관측 모델을 가진 POMDP 와 같습니다 . 또한 각 조종사의 행동을 추론 할 필요가 있습니다. 이것은 미묘한 행동이 될 수 있기 때문에 도전적인 추론 문제입니다 (예 : 위의 경우 주조종사가 controller를 통해 조종하는 것으로 볼 수도 있고 주 조종사가 부조종사에게 행동을 지시하는 것일 수도 있다.).

observation으로 부터 action을 추론하기 위해서는 (s,a) 와 video frame을 연결시키는 어떠한 model이 필요합니다. 그러나 어떤 모델을 선택하는 것은 human value가 그들의 action과 어떻게 관련되어 있는지에 대한 가정을 만들게 됩니다. 

예를 들어 조종사 중의 하나를 누군가 공격하고 그는 공격자를 때림으로써 그 자신을 방어 합니다. 이런 반사적인, 본능적인 반응은 조종사의 value에 유익한 행동입니까? 철학자와 신경과학자는 조종사가 반격을 하기 전에 발생하는 정신적 과정을 고려할 수 있습니다. 

만약 IRL 알고리즘이 off-the-shelf action classifier (off-the-shelf - 선박에서 물건을 끄내듯이.. 즉 기존에 존재하는 classfier를 의미하는듯) 를 사용한다면 이러한 정신적 과정에 관한 가정을 고정시킬 것입니다. 동시에 IRL은 이런 정신적 과정을 배우지 못할 것입니다. - reward, action과 관련된 정신적 과정은 직접적으로 관찰되지 않습니다.

Inferring Policies From Video Frames

IRL을 통해 reward function 을 학습 할때 궁극적인 목표는 reward function을 통해 artificial agent의 행동을 유도 하는 것입니다. 이러한 목표는 IRL이라는 알고리즘이 아닌 다른 알고리즘로도 불립니다. 예를 들어 Apprenticeship Learning 에서의 목표는 인간의 policy로 부터 나온 $MDP\backslash R$ 에서의 가장 좋은 policy를 학습하는 것입니다.($\pi^*$ 은 approxmation function, unknown reward function으로 가정됩니다)  Imitation Learning, 은 인간의 policy와 비슷한 policy 를 학습 하는 것입니다.

IRL 에서처럼 policy search techniques 은 agent의 policy를 나타내는 agent의 action을 알 수있어야 합니다. 그래서 그들은 video or history book 에서 IRL을 학습시키는 것과 같은 일을 합니다. IRL 과 다르게 policy search는 explicit reward function을 가지지 않습니다. 이는 또 다른 additional challenge 입니다. 인간과 AI 는 크게 다른 task에 직면하고 다른 action space를 가지고 있습니다. videos or history books 에서의 대부분의 action들은 인공적인 agent에 의해서는 수행되지 않습니다. 또한 비슷한 task 에서도 이런 일이 일어납니다(ex) 1930년에 운전하는 사람 vs 자율주행 자동자  주어진 environment가 다르기 때문에 자율주행자동차는 1930년에 가서는 제대로 운전을 하지 못한다는 뜻인듯) 즉 이는 AI policy를 향상시키기 위해 인간의 policy를 사용하는 transfer learining 이 필요하다는 것을 의미합니다.

IRL Needs Curated Data

우리는 video or history book 에서 기록된 데이터는 IRL 알고리즘을 적용하기는 어렵다고 말했습니다. 특히 Face book 은 더 어려워 보입니다. 우리는 state (사람에게 보여지는 HTML, or pixel) 과 action( click or scrolling) 을 저장할 수 있습니다. 이는 사실 facebook을 넘어서 컴퓨터에서 수행 할 수 있는 모든 작업으로 확장됩니다. 이는 광범위한 task를 포함하지만 여기에는 분명한 한계점이 있습니다. 세계의 많은 사람들의 컴퓨터 활용 능력은 제한되어 있습니다. 이런 방법으로는 그들의 value를 학습 할 수 없습니다. 더욱더 어떤 행동이 아닌 인간의 preference ( 신체 활동을 넘은 정신적 과정) 을 학습하는 것은 더욱 어렵습니다.

# Information and Biases

인간의 행동은 자신의 preference와 belief 에 달려 있습니다. 선호도와 같은 신념은 절대로 직접 관찰되지 않습니다. narrow task (예 : display에서 좋아하는 사진을 선택하는 task)의 경우 MDP 와 같이 상태를 완전히 인식 한 것으로 인간을 모델링 할 수 있습니다 . 그러나 대부분의 실제 작업에서 인간은 시간이 지남에 따라 정보가 제한적이며 정보가 변경됩니다 (예 : POMDP 또는 RL 문제). 만약 IRL이 인간이 완전한 정보를 가지고 있다고 가정한다면, 그 모델은 mis-specified 된 것이고 인간이 다른 시나리오에서 선호하는 것에 대해 일반화하는 것은 오판 될 수있다. 


(1). 누군가 자신의 집에서 카페로 이동한다고 생각해봅시다. 하지만 그 카페는 이미 닫혀있었어요 만약 그들이 full knowledge를 가지고 있다고 가정한다면 IRL은 카페를 가는것 보다 산책을 하는 것을 선호 할거라고 추론 할 것입니다.

(2) 누군가는 실제로 효과가 없는 약을 복용합니다. 이것은 그들이 그 약이 효과가 있을 것이라는 잘못된 믿음 혹은 잘못된 약을 집었기 때문일 수 있습니다. 각각의 설명들은 preference 에 대한 다른 결론을 이끌어 냅니다.

(3) IRL이 사람의 노트북에서의 키 입력을 추론한다고 가정합시다. 사람은 반복적으로 로그인 암호를 잊어 버리고 다시 설정해야 합니다. 이런 행동은 POMDP 모델로 포착하기가 어렵습니다. IRL 은 사람이 그들의 암호를 재설정 하려고 한다고 추론할 수 있습니다.

위의 (3) 예제는 인간의 망각 때문에 발생합니다. - 매우 짧은 문자열 임에도 발생합니다 이는 인간이 합리적인 Agent로 부터 벗어나는 하나의 방법입니다. 심리학 분야에서는 이런 예외들을 다뤄왔습니다. 밑에서 우리는 이런 예외들을 설명할 것입니다.


Time-inconsistency and Procrastination

Alice 라는 사람의 preference를 추론하는 IRL 이 있습니다. 이 알고리즘의 목표는 앨리스가 tedious task or relaxing 을 할 preference를 추론하는 것입니다. Alice 는 T day 동안 woriking or relaxing 을 한 데이터를 가지고 있습니다.




# Learning Values != Robustly Predicting Human Behaviour

지금까지 설명한 IRL 의 문제점은 Out-of-sample 의 인간의 선택을 예측하는데 있어 서능이 저하됨을 보였습니다. 예를들어 누군가 99일 동안 지루한 작업을 하는 경우 목표에 대한 short-cut 이 제공되는 경우에도 지루한 작업을 계속 할 것으로 예측됩니다. 따라서 목표가 human value가 아닌 단순히 행동을 예측하는 경우에도 mis-specification 은 out-of-sample 에 대한 나쁜 예측을 보일 것입니다.

우리의 목표가 인간의 행동을 예측하는 것이 아닌 인간의  value 를 반영하는 AI 시스템을 만드는 것이라고 해봅시다 이러한 목표 (사람을 예측하고 안전한 AI를 만드는 것) 은 뚜렷한 목표입니다. 다음은 차이를 보여주는 예입니다. long-term smoker인 Bob 을 생각해 봅시다 일반적으로 효과가 있는 금연 치료법이 있어도 bob은 담배를 피웁니다. 어쩌면 밥은 흡연의 건강에 대한 악영향에 대해 부인할 수도 있고 밥은 건강이 나빠지면 담배를 피우지 않을 것이라고 생각합니다. ai 가 bob을 돕는 중이라면 우리는 그가 흡연습관을 막도록 할 것이라고 기대합니다. 하지만 ai system은 다른사람의 가치를 bob에게 강요하지 않습니다. 많은 흡연에 대한 부작용이 주어져도 bob의 value는 바뀌지 않습니다.
